<a href="https://colab.research.google.com/github/youngjooo/GPT2/blob/main/7_4_1_gpt2_finetune_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/GPT/tensorflow-ml-nlp-tf2/7.PRETRAIN_METHOD"

/content/drive/MyDrive/GPT/tensorflow-ml-nlp-tf2/7.PRETRAIN_METHOD


In [ ]:
!pip install --upgrade mxnet gluonnlp
!pip install --upgrade transformers
!pip install --upgrade wget
!pip install --upgrade sentencepiece

     |████████████████████████████████| 1.2 MB 5.2 MB/s 


# GPT2를 활용한 한국어 언어 생성 모델

In [ ]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

아레 실행 커멘드는 gpt_ckpt 폴더가 있지 않은 경우에만 실행해주세요.

In [ ]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

우선, 블러온 모델을 활용해 여기서 사용할 모델 클래스를 구현해보자.

GPT2 모델 클래스는 __init__ 함수에서 TFGPT2LMHeadModel을 생성해서 실행할 수 있게 구현한다.

모델을 출력할 때 GPT2 모델에서 출력하는 값은 튜플 형태로, last_hidden_states, past, hidden_states, attentions 순서로 나타낸다.



*   last_hidden_states
>모델의 마지막 레이어에서 출력한 값을 받는다.

*   past
>모델에 각 레이어에서 연산한 결괏값을 출력한다.
>출력값 past를 활용하면 모델을 가지고 다음 토큰을 예측할 때 연산 속도를 빠르게 해주는 효과가 있다.

* hidden_states
>전체 레이어에 대한 은닉 상태 벡터를 모두 출력한 값이다.

* attention
>전체 레이어에 연산한 어텐션 맵 값을 확인할 수 있다.



In [ ]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
        
    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [ ]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## 사전 학습 모델 문장 생성

사전 학습된 GPT2는 언어 모델을 통해 학습했다. 그렇기 때문에 GPT2는 언어 모델이며 텍스트 생성이 가능하다.

텍스트를 모델에 입력할 수 있도록 토크나이저를 생성한다. 여기서 활용하는 토크나이저는 앞서 버트에서 활용한 토크나이저 방식과 다르다.

In [ ]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

생성한 토크나이저 객체와 사전 학습된 GPT2 모델을 활용해 문장 생성 결과를 확인해보자.

문장 생성은 단어 하나가 주어지면 문장을 만들어주는 방식으로 구현했다.

In [ ]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])


def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)
    
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

In [ ]:
generate_sent('이때', gpt_model, greedy=True)

'이때부터                                                                                                   '

In [ ]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때 배우가 되었다.'

## 소설 텍스트 데이터 전처리하기

미세 조정할 학습 데이터를 구성해 보자. 이번 실습에서 학습할 데이터는 소설 "운수 좋은 날"이다.

In [ ]:
DATA_IN_PATH = './data_in/KOR/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]

In [ ]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

## 소설 텍스트 미세 조정 모델 학습

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [ ]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [ ]:
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

Epoch 1/10
16/16 [==============================] - 170s 9s/step - loss: 4.1199 - accuracy_function: 0.1745 - val_loss: 2.8442 - val_accuracy_function: 0.2215
Epoch 2/10
16/16 [==============================] - 140s 9s/step - loss: 2.8992 - accuracy_function: 0.2470 - val_loss: 2.4641 - val_accuracy_function: 0.2637
Epoch 3/10
16/16 [==============================] - 139s 9s/step - loss: 2.4663 - accuracy_function: 0.2789 - val_loss: 2.3128 - val_accuracy_function: 0.2929
Epoch 4/10
16/16 [==============================] - 137s 9s/step - loss: 2.1896 - accuracy_function: 0.3061 - val_loss: 2.2562 - val_accuracy_function: 0.3162
Epoch 5/10
16/16 [==============================] - 148s 9s/step - loss: 1.9672 - accuracy_function: 0.3270 - val_loss: 2.2350 - val_accuracy_function: 0.3353
Epoch 6/10
16/16 [==============================] - 137s 9s/step - loss: 1.7694 - accuracy_function: 0.3458 - val_loss: 2.2398 - val_accuracy_function: 0.3528
Epoch 7/10
16/16 [============================

In [ ]:
DATA_OUT_PATH = './data_out'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./data_out/tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## 미세 조정 학습 결과 확인

In [ ]:
generate_sent('이때', gpt_model, greedy=True)

'이때                                                                                                    '

In [ ]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때        에  라도                                                                                       '